In [ ]:
# Copyright 2022 Google LLC.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 用于 Python 的 Vertex AI SDK：使用 BigQuery 进行批量预测训练表格回归模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_tabular_regression_batch_bq.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_tabular_regression_batch_bq.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_tabular_regression_batch_bq.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本教程演示了如何使用Python的Vertex AI SDK创建表格回归模型，并使用Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)模型进行批量预测。

了解更多关于[表格数据回归](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview)的信息。

### 目标

在本教程中，您将学习如何使用Python的Vertex AI SDK创建一个AutoML表格回归模型，然后部署它以进行批量预测。您还可以选择使用`gcloud`命令行工具或通过Cloud控制台来创建和部署模型。

本教程使用以下Google Cloud ML服务：

- Vertex AI数据集（表格）
- Vertex AI训练（AutoML表格训练）
- Vertex AI模型注册表
- Vertex AI端点
- Vertex AI批量预测

执行的步骤包括：

- 创建一个Vertex AI `数据集`资源。
- 训练模型。
- 查看模型评估。
- 将`模型`资源部署到一个服务`端点`资源。
- 进行预测。
- 取消部署`模型`。

### 数据集

这个教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)中的[GSOD数据集](https://console.cloud.google.com/marketplace/product/noaa-public/gsod)。在这个数据集的版本中，您可以使用年份、月份和日期字段来预测每日平均温度（mean_temp）。

### 费用

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* 云存储
* BigQuery / BigQuery ML

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)、[云存储
定价](https://cloud.google.com/storage/pricing) 和 [BigQuery 定价](https://cloud.google.com/bigquery/pricing) 并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用情况生成费用估算。

## 安装

安装以下需要执行此笔记本的软件包。

In [ ]:
# Install the packages
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                    'google-cloud-bigquery[bqstorage,pandas]' \
                                    google-cloud-storage 
                                    

### 仅限协作：取消以下单元格的注释以重启内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前
#### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面: [找到项目 ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

### 验证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经通过验证。
**2. 本地 JupyterLab 实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上向您的服务帐号授予云存储权限。

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

#教程

现在你已经准备好开始创建你自己的AutoML表格回归模型。

### BigQuery培训数据的位置。

将`IMPORT_File`变量设置为BigQuery中数据表的位置。

In [ ]:
IMPORT_FILE = "bigquery-public-data.samples.gsod"

准备批量预测数据

从原始数据创建两个数据集。

In [ ]:
# Create client in default region
bq_client = bigquery.Client(
    project=PROJECT_ID,
    credentials=aiplatform.initializer.global_config.credentials,
)

In [ ]:
# Create training dataset in default region
TRAINING_INPUT_DATASET_ID = "gsod_training_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

# Create test dataset in default region
PREDICTION_INPUT_DATASET_ID = "gsod_prediction_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

In [ ]:
# Select top 3000 rows of dataset
TRAINING_SIZE = 3000
query = f"""
        SELECT *
        FROM {IMPORT_FILE}
        LIMIT {TRAINING_SIZE}
        """

TRAINING_INPUT_TABLE_ID = f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}.test"
job_config = bigquery.QueryJobConfig(destination=TRAINING_INPUT_TABLE_ID)

query_job = bq_client.query(query, job_config=job_config)  # API request
query_job.result()  # Waits for query to finish

In [ ]:
# Select a subset of the original dataset for testing
PREDICTION_SIZE = 100
query = f"""
        SELECT *
        FROM {IMPORT_FILE}
        LIMIT {PREDICTION_SIZE}
        OFFSET {TRAINING_SIZE}        
        """

PREDICTION_INPUT_TABLE_ID = f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}.prediction"
job_config = bigquery.QueryJobConfig(destination=PREDICTION_INPUT_TABLE_ID)

query_job = bq_client.query(query, job_config=job_config)  # API request
query_job.result()  # Waits for query to finish

### 创建数据集

使用`TabularDataset.create()`来创建一个`TabularDataset`资源，它接受以下参数：

- `display_name`: `数据集`资源的人类可读名称。
- `gcs_source`: 一个或多个数据集索引文件列表，用于将数据项导入`数据集`资源。
- `bq_source`: 或者，将数据项从BigQuery表导入`数据集`资源。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data_unique",
    bq_source=[f"bq://{TRAINING_INPUT_TABLE_ID}"],
)

label_column = "mean_temp"

print(dataset.resource_name)

In [ ]:
COLUMN_SPECS = {
    "year": "auto",
    "month": "auto",
    "day": "auto",
}

label_column = "mean_temp"

### 创建和运行训练流程

要训练一个AutoML模型，请创建并运行一个训练流程。

#### 创建训练流程

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练流程，包括以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `optimization_prediction_type`：为模型训练指定的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：优化目标（最小化或最大化）。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是训练流程的有向无环图（DAG）。

In [ ]:
training_job = aiplatform.AutoMLTabularTrainingJob(
    display_name="job_unique",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

print(training_job)

#### 运行训练管道

通过使用以下参数调用`run`方法来运行训练作业：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要作为标签训练的列名。
- `budget_milli_node_hours`：（可选）以千分小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则训练可能会在服务认为无法进一步改善模型目标测量之前完成，以节省整个预算。

完成时，`run`方法将返回`Model`资源。

训练管道的执行将需要最多3小时。

In [ ]:
model = training_job.run(
    dataset=dataset,
    model_display_name="model_unique",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 回顾模型评估分数
在模型训练完成后，您可以回顾其评估分数。

In [ ]:
# Get evaluations
model_evaluations = model.list_model_evaluations()

model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

## 发送批量预测请求

现在您可以进行批量预测。

创建一个数据集来存储预测结果。

In [ ]:
# Create results dataset in default region
RESULTS_DATASET_ID = "gsod_results_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{RESULTS_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

### 进行批量预测请求

您可以通过调用batch_predict()方法来进行批量预测，需提供以下参数：

- `job_display_name`：批处理预测作业的可读名称。
- `gcs_source`：一个或多个批处理请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `instances_format`：输入实例的格式，可以是'bigquery'、'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测结果的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作机器人复制品的加速器数量。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

批量预测作业大约需要1小时才能完成。

In [ ]:
# Note: The bigquery_source and bigquery_destination_prefix must be in the same region
PREDICTION_RESULTS_DATASET_ID = f"{PROJECT_ID}.{RESULTS_DATASET_ID}"

batch_predict_job = model.batch_predict(
    job_display_name="tabular_regression_batch_predict_job",
    bigquery_source=f"bq://{PREDICTION_INPUT_TABLE_ID}",
    instances_format="bigquery",
    predictions_format="bigquery",
    bigquery_destination_prefix=f"bq://{PREDICTION_RESULTS_DATASET_ID}",
)

查看批量预测结果

使用BigQuery Python客户端查询目标表，并将结果返回为Pandas dataframe。

In [ ]:
dataframe = (
    bq_client.query(f"SELECT * FROM `{PREDICTION_RESULTS_DATASET_ID}.*`")
    .result()
    .to_dataframe()
)

print(dataframe.head())

清理工作

要清理本项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除以下本教程中创建的单个资源:

- 模型
- AutoML训练作业
- 批处理作业
- Cloud Storage存储桶

In [ ]:
# Delete BigQuery datasets
bq_client.delete_dataset(
    f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}",
    delete_contents=True,
    not_found_ok=True,
)

bq_client.delete_dataset(
    f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}",
    delete_contents=True,
    not_found_ok=True,
)

bq_client.delete_dataset(
    f"{PROJECT_ID}.{RESULTS_DATASET_ID}", delete_contents=True, not_found_ok=True
)

# Delete Vertex AI resources
dataset.delete()
model.delete()
training_job.delete()
batch_predict_job.delete()